In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyomo.environ import *
from pyomo.network import *
from idaes.core.util.model_statistics import degrees_of_freedom
from Method import Initialize
from Unit_Model import *

In [2]:
# Create a concrete model
m = ConcreteModel()

# Define a set for components
m.Phase = Set(initialize = ['Liq','Vap'])
m.Comp = Set(initialize = ['benzene','toluene','methane','hydrogen'])
m.R101 = R101_creater(m)
m.H101 = H101_creater(m)
m.F101 = F101_creater(m)
m.F102 = F102_creater(m)
m.M101 = M101_creater(m)
m.S101 = S101_creater(m)

In [4]:
m.s1 = Arc(source=m.H101.Outlet, destination=m.R101.Inlet)
m.s2 = Arc(source=m.R101.Outlet, destination=m.F101.Inlet)
m.s3 = Arc(source=m.F101.Liq_Outlet, destination=m.F102.Inlet)
m.s4 = Arc(source=m.F101.Vap_Outlet, destination=m.S101.Inlet)
m.s5 = Arc(source=m.S101.Recycle, destination=m.M101.Recycle_Feed)
TransformationFactory("network.expand_arcs").apply_to(m)

In [3]:
# 各组分进料
m.H101.inlet.flow['Liq','benzene'].fix(2.0008e-05)
m.H101.inlet.flow['Liq','toluene'].fix(0.30001)
m.H101.inlet.flow['Liq','methane'].fix(2.0008e-05)
m.H101.inlet.flow['Liq','hydrogen'].fix(2.0008e-05)
m.H101.inlet.flow['Vap','benzene'].fix(0.11934)
m.H101.inlet.flow['Vap','toluene'].fix(0.012508)
m.H101.inlet.flow['Vap','methane'].fix(1.0377)
m.H101.inlet.flow['Vap','hydrogen'].fix(0.56258)

# 温度压力
m.H101.inlet.P.fix(350000)
m.H101.inlet.T.fix(314.09)

m.H101.outlet.P.fix(350000)
m.H101.outlet.T.fix(600)

m.F101.outlet.P.fix(350000)
m.F101.outlet.T.fix(325)

m.F102.outlet.P.fix(150000)
m.F102.outlet.T.fix(375)

m.R101.Q.fix(0)

In [8]:
# 各组分进料
m.M101.inlet_feed['Toluene'].flow['Liq','benzene'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Liq','toluene'].fix(0.30)
m.M101.inlet_feed['Toluene'].flow['Liq','methane'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Liq','hydrogen'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Vap','benzene'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Vap','toluene'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Vap','methane'].fix(1e-8)
m.M101.inlet_feed['Toluene'].flow['Vap','hydrogen'].fix(1e-8)
m.M101.inlet_feed['Toluene'].T.fix(303.2)
m.M101.inlet_feed['Toluene'].P.fix(350000)

m.M101.inlet_feed['Hydrogen'].flow['Liq','benzene'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Liq','toluene'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Liq','methane'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Liq','hydrogen'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Vap','benzene'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Vap','toluene'].fix(1e-8)
m.M101.inlet_feed['Hydrogen'].flow['Vap','methane'].fix(0.02)
m.M101.inlet_feed['Hydrogen'].flow['Vap','hydrogen'].fix(0.30)
m.M101.inlet_feed['Hydrogen'].T.fix(303.2)
m.M101.inlet_feed['Hydrogen'].P.fix(350000)

m.M101.inlet_feed['Recycle'].flow['Liq','benzene'].fix(1e-8)
m.M101.inlet_feed['Recycle'].flow['Liq','toluene'].fix(1e-8)
m.M101.inlet_feed['Recycle'].flow['Liq','methane'].fix(1e-8)
m.M101.inlet_feed['Recycle'].flow['Liq','hydrogen'].fix(1e-8)
m.M101.inlet_feed['Recycle'].flow['Vap','benzene'].fix(0.11932)
m.M101.inlet_feed['Recycle'].flow['Vap','toluene'].fix(0.012488)
m.M101.inlet_feed['Recycle'].flow['Vap','methane'].fix(1.0177)
m.M101.inlet_feed['Recycle'].flow['Vap','hydrogen'].fix(0.26257)
m.M101.inlet_feed['Recycle'].T.fix(325.00)
m.M101.inlet_feed['Recycle'].P.fix(350000)

In [ ]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 3

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

In [6]:
Initialize(m.H101)
Initialize(m.R101,Arc=m.s1)
Initialize(m.F101,Arc=m.s2)
Initialize(m.F102,Arc=m.s3)
Initialize(m.S101,Arc=m.s4)
print('Outlet Flow Rate H101:')
for p in m.Phase:
    for c in m.Comp:
        print('\t [',p,',',c,']: ',value(m.H101.outlet.flow[p,c]))

model.name="F101";
    - termination condition: infeasible
    - message from solver: Ipopt 3.13.2\x3a Converged to a locally infeasible
      point. Problem may be infeasible.
model.name="F102";
    - termination condition: infeasible
    - message from solver: Ipopt 3.13.2\x3a Converged to a locally infeasible
      point. Problem may be infeasible.
Outlet Flow Rate H101:
	 [ Liq , benzene ]:  1e-08
	 [ Liq , toluene ]:  1e-08
	 [ Liq , methane ]:  1e-08
	 [ Liq , hydrogen ]:  1e-08
	 [ Vap , benzene ]:  0.119360008
	 [ Vap , toluene ]:  0.312518
	 [ Vap , methane ]:  1.037720008
	 [ Vap , hydrogen ]:  0.562600008


In [7]:
print('Recycle Flow Rate S101:')
for p in m.Phase:
    for c in m.Comp:
        print('\t [',p,',',c,']: ',value(m.S101.recycle[p,c]))

Recycle Flow Rate S101:
	 [ Liq , benzene ]:  8e-09
	 [ Liq , toluene ]:  8e-09
	 [ Liq , methane ]:  8e-09
	 [ Liq , hydrogen ]:  8e-09
	 [ Vap , benzene ]:  0.11843834587698908
	 [ Vap , toluene ]:  0.012360613240875774
	 [ Vap , methane ]:  1.0163933546148696
	 [ Vap , hydrogen ]:  0.26248897062359083


In [9]:
Initialize(m.M101,m.s5)
print('total Flow Rate M101:')
for p in m.Phase:
    for c in m.Comp:
        print('\t [',p,',',c,']: ',value(m.M101.outlet.flow[p,c]))

total Flow Rate M101:
	 [ Liq , benzene ]:  2.9999999471738364e-08
	 [ Liq , toluene ]:  0.3000000200009588
	 [ Liq , methane ]:  3.000000004132858e-08
	 [ Liq , hydrogen ]:  3.000000004132858e-08
	 [ Vap , benzene ]:  0.11932001999857494
	 [ Vap , toluene ]:  0.012488019843348502
	 [ Vap , methane ]:  1.0377000100000002
	 [ Vap , hydrogen ]:  0.5625700100000979


In [10]:
purity = value(m.F102.outlet.flow['Vap','benzene'])/sum(m.F102.outlet.flow['Vap',c] for c in m.Comp)
value(purity)

0.808968853035663

In [11]:
degrees_of_freedom(m)

9

In [15]:
# 模型求解
solver = SolverFactory('ipopt')
solver.options['tol'] = 1e-4
results = solver.solve(m)
#results = solver.solve(m)

In [17]:
m.M101.pprint()

M101 : Size=1, Index=None, Active=True
    11 Set Declarations
        Comp : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    4 : {'benzene', 'toluene', 'methane', 'hydrogen'}
        Phase : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    2 : {'Liq', 'Vap'}
        cp_ig_1_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : M101.Phase*M101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), ('Liq', 'hydrogen'), ('Vap', 'benzene'), ('Vap', 'toluene'), ('Vap', 'methane'), ('Vap', 'hydrogen')}
        cp_ig_2_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : M101.Phase*M101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'),

In [9]:
for Unit in m.component_data_objects(Block, active=True):
    print(Unit.name)

R101
H101
F101
s1_expanded
s2_expanded
R101.inlet
R101.outlet
H101.inlet
H101.outlet
F101.inlet
F101.outlet
